In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()



In [48]:
dfrm1=spark.read.csv('imdb_top_1000.csv',header=True,inferSchema=True)

In [83]:
dfrm1.printSchema()

root
 |-- Poster_Link: string (nullable = true)
 |-- Series_Title: string (nullable = true)
 |-- Released_Year: string (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- Overview: string (nullable = true)
 |-- Meta_score: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = true)
 |-- Star2: string (nullable = true)
 |-- Star3: string (nullable = true)
 |-- Star4: string (nullable = true)
 |-- No_of_Votes: string (nullable = true)
 |-- Gross: string (nullable = true)



In [49]:
dfrm1.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan F

In [50]:
dfrm1.createOrReplaceTempView('dfrm1')

In [80]:
query="select * from dfrm1 where director='Jeethu Joseph'"

In [81]:
q=spark.sql(query)

In [82]:
q.show()

+--------------------+------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+-------------+--------+-----+------------+------+-----------+-----+
|         Poster_Link|Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|     Director|   Star1|Star2|       Star3| Star4|No_of_Votes|Gross|
+--------------------+------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+-------------+--------+-----+------------+------+-----------+-----+
|https://m.media-a...|    Drishyam|         2013|          U|160 min|Crime, Drama, Thr...|        8.3|A man goes to ext...|      NULL|Jeethu Joseph|Mohanlal|Meena|Asha Sharath|Ansiba|      30722| NULL|
+--------------------+------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+-------------+--------+-----+------------+------+---------